In [ ]:
#getting and working with data
import pandas as pd
import numpy as np
import re
import os
import datetime as dt
import string

from scipy import stats
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.stats.anova import AnovaRM
from statsmodels.graphics.factorplots import interaction_plot

import pingouin as pg
from pingouin import ttest

from CosinorPy import file_parser, cosinor, cosinor1

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn import metrics

#dimensionality reduction
from sklearn.decomposition import PCA

#clustering
from sklearn.cluster import KMeans
import scipy.cluster.hierarchy as shc
from bioinfokit.visuz import cluster

#visualizing results
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

sns.set_context("poster")
sns.set_style("ticks")
sns.set(font_scale=2)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 15000)
pd.set_option('display.max_colwidth', -1)

import warnings; warnings.simplefilter('ignore')
np.set_printoptions(suppress=True)

# Get data

## Get filtered drinking data combined with RFID (already processed in other nb)

In [ ]:
path = '/Users/abbieschindler/Documents/Schindler_Lab/Data/RFID_VDM/polysubstance_paper/data/final/clean/VDM/VDM_poly_final_clean_231103.csv'
data_VDM_start = pd.read_csv(path)
data_VDM_start = pd.DataFrame(data = data_VDM_start)
data_VDM_start['Id'] = data_VDM_start['Sex'] + '_' + data_VDM_start['IdRFID'].astype('str')

print(data_VDM_start.shape)
data_VDM_start.head()

In [ ]:
data_VDM_start.groupby(['IdRFID'])['ld_day'].max()

In [ ]:
#some got taken out on 8th day 
data_VDM_start = data_VDM_start[data_VDM_start['ld_day']<8]
data_VDM_start = data_VDM_start[data_VDM_start['ld_day']>0]
data_VDM_start = data_VDM_start[data_VDM_start['VDM_RFID_timediff']<3]
data_VDM_start = data_VDM_start[data_VDM_start['Cage_N']>1]

data_VDM_start['intake_ml_kg'] = data_VDM_start['value'] / (data_VDM_start['weight_pre']/1000)

data_VDM_start.head()

In [ ]:
hour_to_zeit = {0:19,
                1:20,
                2:21,
                3:22,
                4:23,
                5:24,
                6:1,
                7:2,
                8:3,
                9:4,
                10:5,
                11:6,
                12:7,
                13:8,
                14:9,
                15:10,
                16:11,
                17:12,
                18:13,
                19:14,
                20:15,
                21:16,
                22:17,
                23:18,
               }

data_VDM_start['zeitgeber'] = data_VDM_start['VDM_hour'].map(hour_to_zeit)

In [ ]:
zeit_to_quart = {0:1,
                 1:1,
                 2:1,
                 3:1,
                 4:1,
                 5:1,
                 6:2,
                 7:2,
                 8:2,
                 9:2,
                 10:2,
                 11:2,
                 12:3,
                 13:3,
                 14:3,
                 15:3,
                 16:3,
                 17:3,
                 18:4,
                 19:4,
                 20:4,
                 21:4,
                 22:4,
                 23:4,
                 }

data_VDM_start['zeit_quart'] = data_VDM_start['zeitgeber'].map(zeit_to_quart)

In [ ]:
len(data_VDM_start[data_VDM_start['Sex']=='female']['IdRFID'].unique())

In [ ]:
len(data_VDM_start[data_VDM_start['Sex']=='male']['IdRFID'].unique())

In [ ]:
data_VDM_start.shape

In [ ]:
data_VDM_total_ld_day_hour = data_VDM_start.groupby(['IdRFID', 'Sex', 'substance', 'ld_cycle', 'zeitgeber', 'ld_day',
                                   ])['intake_ml_kg'].sum().reset_index()

print(data_VDM_total_ld_day_hour.shape)
data_VDM_total_ld_day_hour.head()

In [ ]:
substances = ['water', 'EtOH05', 'EtOH10', 'Fent05', 'Fent20']
data_final = pd.DataFrame()

for IdRFID in data_VDM_total_ld_day_hour['IdRFID'].unique():
    print(IdRFID)
    
    d_animal = data_VDM_total_ld_day_hour[data_VDM_total_ld_day_hour['IdRFID']==IdRFID]
    
    for substance in substances:
        
        days_hours_df = pd.DataFrame(0, index=np.arange(1,8), columns=data_VDM_start['zeitgeber'].unique()).unstack().reset_index()
        days_hours_df.columns = ['zeitgeber', 'ld_day', 'ddd']
        days_hours_df['ld_cycle'] = ['light' if x<13 else 'dark' for x in days_hours_df['zeitgeber']]
        days_hours_df['substance'] = substance
        days_hours_df['IdRFID'] = IdRFID
        days_hours_df['Sex'] = d_animal['Sex'].values[0]
        
        x = days_hours_df.merge(d_animal, on=['IdRFID', 'Sex', 'ld_day', 'ld_cycle', 'zeitgeber', 'substance'], 
                                how='left').sort_values(['substance', 'ld_day', 'zeitgeber'])
        
        if data_final.empty:
            data_final = x
        else:
            data_final = pd.concat([data_final, x], axis=0)
        
    print(data_final.shape,'\n')

data_final = data_final.fillna(0).sort_values(['IdRFID', 'substance', 'ld_day', 'zeitgeber'])
data_final.head()

In [ ]:
zeit_to_quart = {0:1,
                 1:1,
                 2:1,
                 3:1,
                 4:1,
                 5:1,
                 6:2,
                 7:2,
                 8:2,
                 9:2,
                 10:2,
                 11:2,
                 12:3,
                 13:3,
                 14:3,
                 15:3,
                 16:3,
                 17:3,
                 18:4,
                 19:4,
                 20:4,
                 21:4,
                 22:4,
                 23:4,
                 }

data_final['zeit_quart'] = data_final['zeitgeber'].map(zeit_to_quart)

data_final.head()

## Get RFID data (already processed in other nb)

In [ ]:
path = '/Users/abbieschindler/Documents/Schindler_Lab/Data/RFID_VDM/polysubstance_paper/data/final/clean/RFID/RFID_poly_final_clean_231103.csv'

data_RFID_start = pd.read_csv(path)
data_RFID_start = pd.DataFrame(data = data_RFID_start)
print(data_RFID_start.shape)
data_RFID_start.head()

In [ ]:
data_RFID_start.groupby(['IdRFID'])['ld_day'].max()

In [ ]:
#some got taken out on 8th day so end at lights on day 8
data_RFID_start = data_RFID_start[data_RFID_start['ld_day']<8]
data_RFID_start = data_RFID_start[data_RFID_start['ld_day']>0]

data_RFID_start = data_RFID_start[data_RFID_start['Cage_N']>1]
print(data_RFID_start.shape)
data_RFID_start.head()

In [ ]:
hour_to_zeit = {0:19,
                1:20,
                2:21,
                3:22,
                4:23,
                5:24,
                6:1,
                7:2,
                8:3,
                9:4,
                10:5,
                11:6,
                12:7,
                13:8,
                14:9,
                15:10,
                16:11,
                17:12,
                18:13,
                19:14,
                20:15,
                21:16,
                22:17,
                23:18,
               }

data_RFID_start['zeitgeber'] = data_RFID_start['RFID_hour'].map(hour_to_zeit)

In [ ]:
zeit_to_quart = {0:1,
                 1:1,
                 2:1,
                 3:1,
                 4:1,
                 5:1,
                 6:2,
                 7:2,
                 8:2,
                 9:2,
                 10:2,
                 11:2,
                 12:3,
                 13:3,
                 14:3,
                 15:3,
                 16:3,
                 17:3,
                 18:4,
                 19:4,
                 20:4,
                 21:4,
                 22:4,
                 23:4,
                 }

data_RFID_start['zeit_quart'] = data_RFID_start['zeitgeber'].map(zeit_to_quart)

data_RFID_start.head()

## Confirm matching RFIDs 

In [ ]:
set(data_RFID_start['IdRFID'].unique()) - set(data_VDM_start['IdRFID'].unique())

In [ ]:
set(data_VDM_start['IdRFID'].unique()) - set(data_RFID_start['IdRFID'].unique())

In [ ]:
sub_map = {'etoh_05': 'EtOH05',
           'etoh_10': 'EtOH10',
           'fent_05': 'Fent05',
           'fent_20': 'Fent20',
           'water_1': 'water',
           'water_2': 'water',}

data_RFID_start['substance'] = data_RFID_start['unitLabel_drink'].map(sub_map)

In [ ]:
data_RFID_total_ld_day_hour = data_RFID_start.groupby(['IdRFID', 'Sex', 'substance', 'ld_cycle', 'zeitgeber', 'ld_day',
                                   ])['eventDuration'].sum().reset_index()

print(data_RFID_total_ld_day_hour.shape)
data_RFID_total_ld_day_hour.head()

In [ ]:
substances = ['water', 'EtOH05', 'EtOH10', 'Fent05', 'Fent20']
data_RFID_final = pd.DataFrame()

for IdRFID in data_RFID_total_ld_day_hour['IdRFID'].unique():
    print(IdRFID)
    
    d_animal = data_RFID_total_ld_day_hour[data_RFID_total_ld_day_hour['IdRFID']==IdRFID]
    
    for substance in substances:
        
        days_hours_df = pd.DataFrame(0, index=np.arange(1,8), columns=data_RFID_start['zeitgeber'].unique()).unstack().reset_index()
        days_hours_df.columns = ['zeitgeber', 'ld_day', 'ddd']
        days_hours_df['ld_cycle'] = ['light' if x<13 else 'dark' for x in days_hours_df['zeitgeber']]
        days_hours_df['substance'] = substance
        days_hours_df['IdRFID'] = IdRFID
        days_hours_df['Sex'] = d_animal['Sex'].values[0]
        
        x = days_hours_df.merge(d_animal, on=['IdRFID', 'Sex', 'ld_day', 'ld_cycle', 'zeitgeber', 'substance'], 
                                how='left').sort_values(['substance', 'ld_day', 'zeitgeber'])
        
        if data_RFID_final.empty:
            data_RFID_final = x
        else:
            data_RFID_final = pd.concat([data_RFID_final, x], axis=0)
        
    print(data_RFID_final.shape,'\n')

data_RFID_final = data_RFID_final.fillna(0).sort_values(['IdRFID', 'substance', 'ld_day', 'zeitgeber'])
data_RFID_final.head()

In [ ]:
zeit_to_quart = {0:1,
                 1:1,
                 2:1,
                 3:1,
                 4:1,
                 5:1,
                 6:2,
                 7:2,
                 8:2,
                 9:2,
                 10:2,
                 11:2,
                 12:3,
                 13:3,
                 14:3,
                 15:3,
                 16:3,
                 17:3,
                 18:4,
                 19:4,
                 20:4,
                 21:4,
                 22:4,
                 23:4,
                 }

data_RFID_final['zeit_quart'] = data_RFID_final['zeitgeber'].map(zeit_to_quart)

data_RFID_final.head()

# RFID data

In [ ]:
data_RFID = data_RFID_final

## total with combined substances

### event duration proportion

In [ ]:
sns.displot(data=data_RFID_start, x="eventDuration", kind='ecdf', hue='Sex', palette='PuBuGn')
#plt.ylim(0,2000)
plt.xlim(0,30)
plt.xlabel('Time in chamber (sec)')

m=data_RFID_start[data_RFID_start['Sex']=='male']['eventDuration'].values
f=data_RFID_start[data_RFID_start['Sex']=='female']['eventDuration'].values
print(stats.kstest(m, f))

plt.show()

### total event duration sum

In [ ]:
data_RFID_total = data_RFID.groupby(['IdRFID', 'Sex', 
                                   ])['eventDuration'].sum().reset_index()

data_RFID_total['Time in chamber (min)'] = data_RFID_total['eventDuration']/60

data_RFID_total.head()

In [ ]:
g = sns.catplot(x='Sex', y='Time in chamber (min)', data=data_RFID_total,  kind='bar', height=5, aspect=1,
            ci=68, palette='PuBuGn')

# map data to stripplot
g.map(sns.stripplot, 'Sex', 'Time in chamber (min)', 
      palette=sns.color_palette('PuBuGn')[2:4], dodge=True, alpha=0.6, ec='k', linewidth=1)
plt.ylim(0,2000)
plt.show()

male = data_RFID_total[data_RFID_total['Sex']=='male']['Time in chamber (min)']
female = data_RFID_total[data_RFID_total['Sex']=='female']['Time in chamber (min)']
stats.ttest_ind(female, male)

## light dark with combined substances

### event duration proportion

In [ ]:
g=sns.displot(data=data_RFID_start, x="eventDuration", kind='ecdf', hue='ld_cycle', col='Sex', palette='PuBuGn')
#plt.ylim(0,2000)
plt.xlim(0,30)

g.set_axis_labels('Time in chamber (sec)')
plt.show()

### total event duration sum

In [ ]:
data_RFID_total_ld = data_RFID.groupby(['IdRFID', 'Sex', 'ld_cycle', 
                                   ])['eventDuration'].sum().reset_index()

data_RFID_total_ld['Time in chamber (min)'] = data_RFID_total_ld['eventDuration']/60

data_RFID_total_ld.head()

In [ ]:
g= sns.catplot(x='ld_cycle', y='Time in chamber (min)', data=data_RFID_total_ld,  kind='bar', height=5, aspect=1,
            ci=68, hue='Sex', palette='PuBuGn')

# map data to stripplot
g.map(sns.stripplot, 'ld_cycle', 'Time in chamber (min)', 'Sex',
      palette=sns.color_palette('PuBuGn')[2:4], dodge=True, alpha=0.6, ec='k', linewidth=1)
plt.ylim(0,2000)
plt.xlabel('Light/Dark Cycle')
plt.show()

In [ ]:
dv = 'Time in chamber (min)'

#RM using Pingouin
aov = pg.mixed_anova(dv=dv, within='ld_cycle', between='Sex', subject='IdRFID', data=data_RFID_total_ld)
pg.print_table(aov)

#posthocs
posthocs = pg.pairwise_ttests(dv=dv, within='ld_cycle', between='Sex',
                              subject='IdRFID', data=data_RFID_total_ld, padjust='bonf')
pg.print_table(posthocs)

## daily total with combined substance 

### event duration proportion

In [ ]:
g=sns.displot(data=data_RFID_start, x="eventDuration", kind='ecdf', hue='ld_day', col='Sex', palette='PuBuGn')
#plt.ylim(0,2000)
plt.xlim(0,30)

g.set_axis_labels('Time in chamber (sec)')
plt.show()

### total event duration sum

In [ ]:
data_RFID_total_daily = data_RFID.groupby(['IdRFID', 'Sex', 'ld_day',
                                   ])['eventDuration'].sum().reset_index()

data_RFID_total_daily['Time in chamber (min)'] = data_RFID_total_daily['eventDuration']/60

data_RFID_total_daily.head()

In [ ]:
g= sns.catplot(x='ld_day', y='Time in chamber (min)', data=data_RFID_total_daily,  kind='bar', height=5, aspect=2,
            ci=68, hue='Sex', palette='PuBuGn')

# map data to stripplot
g.map(sns.stripplot, 'ld_day', 'Time in chamber (min)', 'Sex',
      palette=sns.color_palette('PuBuGn')[2:4], dodge=True, alpha=0.6, ec='k', linewidth=1)

plt.xlabel('Day')
plt.show()

In [ ]:
dv = 'Time in chamber (min)'

#RM using Pingouin
aov = pg.mixed_anova(dv=dv, within='ld_day', between='Sex', subject='IdRFID', data=data_RFID_total_daily)
pg.print_table(aov)

#posthocs
posthocs = pg.pairwise_ttests(dv=dv, within='ld_day', between='Sex',
                              subject='IdRFID', data=data_RFID_total_daily, padjust='bonf')
pg.print_table(posthocs)

## hourly total with combined substances

### event duration proportion

In [ ]:
g=sns.displot(data=data_RFID_start, x="eventDuration", kind='ecdf', hue='zeitgeber', col='Sex', palette='PuBuGn')
#plt.ylim(0,2000)
plt.xlim(0,30)

g.set_axis_labels('Time in chamber (sec)')
plt.show()

### total event duration sum

In [ ]:
data_RFID_total_hourly = data_RFID.groupby(['IdRFID', 'Sex', 'zeitgeber', 
                                   ])['eventDuration'].sum().reset_index()

data_RFID_total_hourly['Time in chamber (min)'] = data_RFID_total_hourly['eventDuration']/60

data_RFID_total_hourly.head()

In [ ]:
g= sns.catplot(x='zeitgeber', y='Time in chamber (min)', data=data_RFID_total_hourly,  kind='bar', height=5, aspect=2,
            ci=68, hue='Sex', palette='PuBuGn')

# map data to stripplot
g.map(sns.stripplot, 'zeitgeber', 'Time in chamber (min)', 'Sex',
      palette=sns.color_palette('PuBuGn')[2:4], dodge=True, alpha=0.6, ec='k', linewidth=1,
      )

plt.xlabel('Zeitgeber time (hour)')
plt.show()

In [ ]:
dv = 'Time in chamber (min)'

#RM using Pingouin
aov = pg.mixed_anova(dv=dv, within='zeitgeber', between='Sex', subject='IdRFID', data=data_RFID_total_hourly)
pg.print_table(aov)

#posthocs
posthocs = pg.pairwise_ttests(dv=dv, within='zeitgeber', between='Sex',
                              subject='IdRFID', data=data_RFID_total_hourly, padjust='bonf')
pg.print_table(posthocs)

## daily and hourly combined substances heatmap

In [ ]:
data_RFID_total_hour_day = data_RFID.groupby(['IdRFID', 'Sex', 'zeitgeber', 'ld_day',
                                   ])['eventDuration'].sum().reset_index()

data_RFID_total_hour_day['Time in chamber (min)'] = data_RFID_total_hour_day['eventDuration']/60

data_RFID_total_hour_day.head()

### event duration

In [ ]:
d = data_RFID_total_hour_day[data_RFID_total_hour_day['Sex']=='female']
groupby = d.groupby(['ld_day', 'zeitgeber'])['Time in chamber (min)'].mean().reset_index()


sns.set(font_scale=2)
groupby = groupby.pivot('ld_day', 'zeitgeber', "Time in chamber (min)")
plt.figure(figsize=(5,5))
ax = sns.heatmap(groupby, cmap="PuBuGn", linewidths=.25, vmax=12,
                     cbar_kws={'label': 'Time in chamber (min)'})

#plt.xlabel('Bottle')
plt.show()

## total with separate substances 

### event duration proportion

In [ ]:
order=['water', 'EtOH05', 'EtOH10', 'Fent05', 'Fent20']

g=sns.displot(data=data_RFID_start, x="eventDuration", kind='ecdf', hue='substance', col='Sex', palette='PuBuGn',
             hue_order=order)
#plt.ylim(0,2000)
plt.xlim(0,30)

g.set_axis_labels('Time in chamber (sec)')
plt.show()

### total event duration sum

In [ ]:
order=['water', 'EtOH05', 'EtOH10', 'Fent05', 'Fent20']
data_RFID_total_sub = data_RFID.groupby(['IdRFID', 'Sex', 'substance',
                                   ])['eventDuration'].sum().reset_index()

data_RFID_total_sub['Time in chamber (min)'] = data_RFID_total_sub['eventDuration']/60

data_RFID_total_sub.head()

In [ ]:
g= sns.catplot(x='substance', y='Time in chamber (min)', data=data_RFID_total_sub,  kind='bar', height=5, aspect=2,
            ci=68, hue='Sex', order=order, palette='PuBuGn')

# map data to stripplot
g.map(sns.stripplot, 'substance', 'Time in chamber (min)', 'Sex',
      palette=sns.color_palette('PuBuGn')[2:4], dodge=True, alpha=0.6, ec='k', linewidth=1,
      order=order)

plt.xlabel('Chamber substance')
plt.show()

In [ ]:
dv = 'Time in chamber (min)'

#RM using Pingouin
aov = pg.mixed_anova(dv=dv, within='substance', between='Sex', subject='IdRFID', data=data_RFID_total_sub)
pg.print_table(aov)

#posthocs
posthocs = pg.pairwise_ttests(dv=dv, within='substance', between='Sex',
                              subject='IdRFID', data=data_RFID_total_sub, padjust='bonf')
pg.print_table(posthocs)

## daily with separate substances 

### event duration proportion

In [ ]:
d = data_RFID_start[data_RFID_start['substance']=='water']

g=sns.displot(data=d, x="eventDuration", kind='ecdf', hue='ld_day', col='Sex', palette='PuBuGn')
#plt.ylim(0,2000)
plt.xlim(0,30)

g.set_axis_labels('Time in chamber (sec)')
plt.show()

### total event duration sum

In [ ]:
data_RFID_sub_daily = data_RFID.groupby(['IdRFID', 'Sex', 'ld_day', 'substance'
                                   ])['eventDuration'].sum().reset_index()

data_RFID_sub_daily['Time in chamber (min)'] = data_RFID_sub_daily['eventDuration']/60

data_RFID_sub_daily.head()

In [ ]:
sub = 'Fent20'
d = data_RFID_sub_daily[data_RFID_sub_daily['substance']==sub]

g= sns.catplot(x='Sex', y='Time in chamber (min)', data=d,  kind='bar', height=5, aspect=1,
            ci=68, hue='ld_day', palette='PuBuGn', )

# map data to stripplot
g.map(sns.stripplot, 'Sex', 'Time in chamber (min)', 'ld_day', 
      palette=sns.color_palette('PuBuGn')[2:4], dodge=True, alpha=0.6, ec='k', linewidth=1 )

plt.ylim(0,200)
plt.title(sub)
plt.xlabel('Day')
plt.show()

In [ ]:
dv = 'Time in chamber (min)'

d = data_RFID_sub_daily[data_RFID_sub_daily['substance']==sub]

#RM using Pingouin
aov = pg.mixed_anova(dv=dv, within='ld_day', between='Sex', subject='IdRFID', data=d)
pg.print_table(aov)

#posthocs
posthocs = pg.pairwise_ttests(dv=dv, within='ld_day', between='Sex',
                              subject='IdRFID', data=d, padjust='fdr_bh')
pg.print_table(posthocs)

## light dark with separate substances 

### event duration proportion

In [ ]:
d = data_RFID_start[data_RFID_start['Sex']=='male']

g=sns.displot(data=d, x="eventDuration", kind='ecdf', hue='substance', col='ld_cycle', palette='PuBuGn',
              hue_order=order)
#plt.ylim(0,2000)
plt.xlim(0,30)

g.set_axis_labels('Time in chamber (sec)')
plt.show()

### total event duration sum

In [ ]:
data_RFID_sub_ld = data_RFID.groupby(['IdRFID', 'Sex', 'ld_cycle', 'substance',
                                   ])['eventDuration'].sum().reset_index()

data_RFID_sub_ld['Time in chamber (min)'] = data_RFID_sub_ld['eventDuration']/60

data_RFID_sub_ld.head()

In [ ]:
d = data_RFID_sub_ld[data_RFID_sub_ld['Sex']=='female']

g= sns.catplot(x='ld_cycle', y='Time in chamber (min)', data=d,  kind='bar', height=5, aspect=1,
            ci=68, hue='substance', palette='PuBuGn', hue_order=order)

# map data to stripplot
g.map(sns.stripplot, 'ld_cycle', 'Time in chamber (min)', 'substance', 
      palette=sns.color_palette('PuBuGn')[2:4], dodge=True, alpha=0.6, ec='k', linewidth=1 , hue_order=order)

plt.ylim(0,500)
plt.xlabel('Light/Dark Cycle')
plt.show()

In [ ]:
data_RFID_sub_ld.set_index(['IdRFID', 'Sex', 'ld_cycle', 'substance'])['Time in chamber (min)'].unstack(-1).to_csv('data_RFID_sub_ld.csv')

## daily and hourly total with separate substance heatmap

### event duration

In [ ]:
data_RFID_total_sub_day_hour = data_RFID.groupby(['IdRFID', 'Sex', 'substance', 'ld_day', 'zeitgeber',
                                   ])['eventDuration'].sum().reset_index()

data_RFID_total_sub_day_hour['Time in chamber (min)'] = data_RFID_total_sub_day_hour['eventDuration']/60

data_RFID_total_sub_day_hour.head()

In [ ]:
d = data_RFID_total_sub_day_hour[data_RFID_total_sub_day_hour['Sex']=='female']
d = d[d['substance']=='Fent20']
groupby = d.groupby(['ld_day', 'zeitgeber'])['Time in chamber (min)'].mean().reset_index()


sns.set(font_scale=2)
groupby = groupby.pivot('ld_day', 'zeitgeber', "Time in chamber (min)")
plt.figure(figsize=(5,5))
ax = sns.heatmap(groupby, cmap="PuBuGn", linewidths=.25, vmax=5,
                     cbar_kws={'label': 'Time in chamber (min)'})

#plt.xlabel('Bottle')
plt.show()

## ID heatmap

In [ ]:
data_RFID_total_sub['Id'] = data_RFID_total_sub['Sex'] + '_' + data_RFID_total_sub['IdRFID'].astype('str')
data_RFID_total_sub.head()

In [ ]:
sns.set(font_scale=2)
groupby = data_RFID_total_sub.pivot('Id', 'substance', "Time in chamber (min)")
plt.figure(figsize=(5,30))
ax = sns.heatmap(groupby, cmap="PuBuGn", linewidths=.25, vmax=600,
                     cbar_kws={'label': 'Time in chamber (min)'})


#plt.xlabel('Bottle')
plt.show()

## Raster

In [ ]:
mice = ['041AC1620E', '041AC15DE9', '041AC16321', '041AC16A00', ]
data_idivid = data_RFID_start[data_RFID_start['IdRFID']==mice[3]]
data_idivid = data_idivid[data_idivid['ld_day']==4]
colorCodes = ['C{}'.format(i) for i in range(5)]

param = 'RFID_sec_count_running'
data_1 = data_idivid[data_idivid['substance'] == 'water'][param].values
data_2 = data_idivid[data_idivid['substance'] == 'EtOH05'][param].values
data_3 = data_idivid[data_idivid['substance'] == 'EtOH10'][param].values
data_4 = data_idivid[data_idivid['substance'] == 'Fent05'][param].values
data_5 = data_idivid[data_idivid['substance'] == 'Fent20'][param].values

data_array = np.array([data_1, data_2, data_3, data_4, data_5])

# Draw a spike raster plot
plt.figure(figsize=(19,5))
params = {'axes.labelsize': 75,
          'axes.titlesize': 75}
plt.rc('xtick', labelsize=40) 
plt.rc('ytick', labelsize=40) 
plt.rcParams.update(params)
plt.yticks([0,1,2,3,4], 
           ['water', 'EtOH05', 'EtOH10', 'Fent05', 'Fent20'])
plt.eventplot(data_array, color=colorCodes, linelengths = 0.3) 
plt.xlabel('Time (running second count)', fontsize=50)
plt.ylabel('Drinking chamber', fontsize=50)

plt.show()

# Drinking data

In [ ]:
sns.set_context("poster")
sns.set_style("ticks")
sns.set(font_scale=2)

## total with combined substances

In [ ]:
data_final_clean_3s = data_final
data_final_clean_3s.head()

### total intake sum min

In [ ]:
data_VDM_total_min = data_VDM_start.groupby(['IdRFID', 'Sex', 'VDM_min_count_running',
                                   ])[['intake_ml_kg']].sum().reset_index()

data_VDM_total_min.head()

In [ ]:
sns.displot(data=data_VDM_total_min, x="intake_ml_kg", kind='ecdf', hue='Sex',
           palette='PuBuGn')

plt.xlabel('Intake (ml/kg) per minute')

plt.xlim(0,10)

### total intake sum

In [ ]:
data_VDM_total = data_final_clean_3s.groupby(['IdRFID', 'Sex', 
                                   ])[['intake_ml_kg']].sum().reset_index()

data_VDM_total.head()

In [ ]:
g = sns.catplot(x='Sex', y='intake_ml_kg', data=data_VDM_total,  kind='bar', height=5, aspect=1,
            ci=68, palette='PuBuGn')

# map data to stripplot
g.map(sns.stripplot, 'Sex', 'intake_ml_kg', 
      palette=sns.color_palette('PuBuGn')[2:4], dodge=True, alpha=0.6, ec='k', linewidth=1)
plt.ylabel('Total intake (ml/kg)')
plt.show()

male = data_VDM_total[data_VDM_total['Sex']=='male']['intake_ml_kg']
female = data_VDM_total[data_VDM_total['Sex']=='female']['intake_ml_kg']
stats.ttest_ind(female, male)

## light dark with combined substances

### total intake sum min

In [ ]:
data_VDM_total_ld_min = data_VDM_start.groupby(['IdRFID', 'Sex', 'VDM_min_count_running', 'ld_cycle',
                                   ])[['intake_ml_kg']].sum().reset_index()

data_VDM_total_ld_min.head()

In [ ]:
g=sns.displot(data=data_VDM_total_ld_min, x="intake_ml_kg", kind='ecdf', hue='ld_cycle', col='Sex', palette='PuBuGn')
#plt.ylim(0,2000)
plt.xlim(0,10)

g.set_axis_labels('Intake (ml/kg) per minute')
plt.show()

### total intake sum

In [ ]:
data_VDM_total_ld = data_final_clean_3s.groupby(['IdRFID', 'Sex', 'ld_cycle',
                                   ])[['intake_ml_kg']].sum().reset_index()

data_VDM_total_ld.head()

In [ ]:
g= sns.catplot(x='ld_cycle', y='intake_ml_kg', data=data_VDM_total_ld,  kind='bar', height=5, aspect=1,
            ci=68, hue='Sex', palette='PuBuGn')

# map data to stripplot
g.map(sns.stripplot, 'ld_cycle', 'intake_ml_kg', 'Sex',
      palette=sns.color_palette('PuBuGn')[2:4], dodge=True, alpha=0.6, ec='k', linewidth=1)


plt.xlabel('Light/Dark Cycle')
plt.ylabel('Total intake (ml/kg)')
plt.show()

In [ ]:
dv = 'intake_ml_kg'

#RM using Pingouin
aov = pg.mixed_anova(dv=dv, within='ld_cycle', between='Sex', subject='IdRFID', data=data_VDM_total_ld)
pg.print_table(aov)

#posthocs
posthocs = pg.pairwise_ttests(dv=dv, within='ld_cycle', between='Sex',
                              subject='IdRFID', data=data_VDM_total_ld, padjust='holm')
pg.print_table(posthocs)

## daily total with combined substances

### total intake sum min

In [ ]:
data_VDM_total_daily_min = data_VDM_start.groupby(['IdRFID', 'Sex', 'VDM_min_count_running', 'ld_day',
                                   ])[['intake_ml_kg']].sum().reset_index()

data_VDM_total_daily_min.head()

In [ ]:
g=sns.displot(data=data_VDM_total_daily_min, x="intake_ml_kg", kind='ecdf', hue='ld_day', col='Sex', palette='PuBuGn')
#plt.ylim(0,2000)
plt.xlim(0,10)

g.set_axis_labels('Intake (ml/kg) per minute')
plt.show()

### total intake sum

In [ ]:
data_VDM_total_daily = data_final_clean_3s.groupby(['IdRFID', 'Sex', 'ld_day',
                                   ])[['intake_ml_kg']].sum().reset_index()

data_VDM_total_daily.head()

In [ ]:
g= sns.catplot(x='ld_day', y='intake_ml_kg', data=data_VDM_total_daily,  kind='bar', height=5, aspect=2,
            ci=68, hue='Sex', palette='PuBuGn')

# map data to stripplot
g.map(sns.stripplot, 'ld_day', 'intake_ml_kg', 'Sex',
      palette=sns.color_palette('PuBuGn')[2:4], dodge=True, alpha=0.6, ec='k', linewidth=1)

plt.xlabel('Day')
plt.ylabel('Total intake (ml/kg)')
plt.show()

In [ ]:
dv = 'intake_ml_kg'

#RM using Pingouin
aov = pg.mixed_anova(dv=dv, within='ld_day', between='Sex', subject='IdRFID', data=data_VDM_total_daily)
pg.print_table(aov)

#posthocs
posthocs = pg.pairwise_ttests(dv=dv, within='ld_day', between='Sex',
                              subject='IdRFID', data=data_VDM_total_daily, padjust='holm')
pg.print_table(posthocs)

## hourly total with combined substances

### total intake sum

In [ ]:
data_VDM_total_zeit = data_final_clean_3s.groupby(['IdRFID', 'Sex', 'zeitgeber',
                                   ])[['intake_ml_kg']].sum().reset_index()

data_VDM_total_zeit.head()

In [ ]:
g= sns.catplot(x='zeitgeber', y='intake_ml_kg', data=data_VDM_total_zeit,  kind='bar', height=5, aspect=2,
            ci=68, hue='Sex', palette='PuBuGn')

# map data to stripplot
g.map(sns.stripplot, 'zeitgeber', 'intake_ml_kg', 'Sex',
      palette=sns.color_palette('PuBuGn')[2:4], dodge=True, alpha=0.6, ec='k', linewidth=1)

plt.xlabel('Zeitgeber time (hour)')
plt.ylabel('Total intake (ml/kg)')
plt.show()

In [ ]:
dv = 'intake_ml_kg'

#RM using Pingouin
aov = pg.mixed_anova(dv=dv, within='zeitgeber', between='Sex', subject='IdRFID', data=data_VDM_total_zeit)
pg.print_table(aov)

#posthocs
posthocs = pg.pairwise_ttests(dv=dv, within='zeitgeber', between='Sex',
                              subject='IdRFID', data=data_VDM_total_zeit, padjust='bonf')
pg.print_table(posthocs)

## daily and hourly combines substances heatmap

### total drinking

In [ ]:
data_VDM_total_hour_day = data_final_clean_3s.groupby(['IdRFID', 'Sex', 'zeitgeber', 'ld_day',
                                   ])['intake_ml_kg'].sum().reset_index()

data_VDM_total_hour_day.head()

In [ ]:
d = data_VDM_total_hour_day[data_VDM_total_hour_day['Sex']=='female']
groupby = d.groupby(['ld_day', 'zeitgeber'])['intake_ml_kg'].mean().reset_index()


sns.set(font_scale=2)
groupby = groupby.pivot('ld_day', 'zeitgeber', "intake_ml_kg")
plt.figure(figsize=(5,5))
ax = sns.heatmap(groupby, cmap="PuBuGn", linewidths=.25, vmax=15,
                     cbar_kws={'label': 'Total intake (ml/kg)'})

#plt.xlabel('Bottle')
plt.show()

## total with separate substances

### total intake sum min

In [ ]:
data_VDM_total_sub_min = data_VDM_start.groupby(['IdRFID', 'Sex', 'VDM_min_count_running', 'substance',
                                   ])[['intake_ml_kg']].sum().reset_index()

data_VDM_total_sub_min.head()

In [ ]:
g=sns.displot(data=data_VDM_total_sub_min, x="intake_ml_kg", kind='ecdf', hue='substance', col='Sex', 
              hue_order=order, palette='PuBuGn')
#plt.ylim(0,2000)
plt.xlim(0,10)

g.set_axis_labels('Intake (ml/kg) per minute')
plt.show()

### total intake sum

In [ ]:
data_VDM_total_sub = data_final_clean_3s.groupby(['IdRFID', 'Sex', 'substance',
                                   ])[['intake_ml_kg']].sum().reset_index()

data_VDM_total_sub.tail()

In [ ]:
g= sns.catplot(x='substance', y='intake_ml_kg', data=data_VDM_total_sub,  kind='bar', height=5, aspect=2,
            ci=68, hue='Sex', order=order, palette='PuBuGn')

# map data to stripplot
g.map(sns.stripplot, 'substance', 'intake_ml_kg', 'Sex',
      palette=sns.color_palette('PuBuGn')[2:4], dodge=True, alpha=0.6, ec='k', linewidth=1,
      order=order,)

plt.xlabel('Chamber substance')
plt.ylabel('Total intake (ml/kg)')
plt.show()

In [ ]:
dv = 'intake_ml_kg'

#RM using Pingouin
aov = pg.mixed_anova(dv=dv, within='substance', between='Sex', subject='IdRFID', data=data_VDM_total_sub)
pg.print_table(aov)

#posthocs
posthocs = pg.pairwise_ttests(dv=dv, within='substance', between='Sex',
                              subject='IdRFID', data=data_VDM_total_sub, padjust='bonf')
pg.print_table(posthocs)

## light dark with separate substances 

### total intake

In [ ]:
data_VDM_total_sub_ld = data_final_clean_3s.groupby(['IdRFID', 'Sex', 'ld_cycle', 'substance',
                                   ])[['intake_ml_kg']].sum().reset_index()

data_VDM_total_sub_ld.head()

In [ ]:
d = data_VDM_total_sub_ld[data_VDM_total_sub_ld['Sex']=='female']

g= sns.catplot(x='ld_cycle', y='intake_ml_kg', data=d,  kind='bar', height=5, aspect=1,
            ci=68, hue='substance', palette='PuBuGn', hue_order=order)

# map data to stripplot
g.map(sns.stripplot, 'ld_cycle', 'intake_ml_kg', 'substance', hue_order=order,
      palette=sns.color_palette('PuBuGn')[2:4], dodge=True, alpha=0.6, ec='k', linewidth=1)

plt.ylim(0,1250)
plt.xlabel('Light/Dark Cycle')
plt.ylabel('Total intake (ml/kg)')
plt.show()

In [ ]:
data_VDM_total_sub_ld.set_index(['IdRFID', 'Sex', 'ld_cycle', 'substance'])['intake_ml_kg'].unstack(-1).to_csv('data_VDM_total_sub_ld.csv')

## daily with separate substances 

In [ ]:
data_VDM_total_sub_day = data_final_clean_3s.groupby(['IdRFID', 'Sex', 'ld_day', 'substance',
                                   ])[['intake_ml_kg']].sum().reset_index()

data_VDM_total_sub_day.head()

In [ ]:
sub = 'Fent20'
d = data_VDM_total_sub_day[data_VDM_total_sub_day['substance']==sub]

g= sns.catplot(x='Sex', y='intake_ml_kg', data=d,  kind='bar', height=5, aspect=1,
            ci=68, hue='ld_day', palette='PuBuGn')

# map data to stripplot
g.map(sns.stripplot, 'Sex', 'intake_ml_kg', 'ld_day', 
      palette=sns.color_palette('PuBuGn')[2:4], dodge=True, alpha=0.6, ec='k', linewidth=1)

plt.ylim(0,250)
plt.title(sub)
plt.xlabel('Day')
plt.ylabel('Daily intake (ml/kg)')
plt.show()

In [ ]:
dv = 'intake_ml_kg'

#RM using Pingouin
aov = pg.mixed_anova(dv=dv, within='ld_day', between='Sex', subject='IdRFID', data=d)
pg.print_table(aov)

#posthocs
posthocs = pg.pairwise_ttests(dv=dv, within='ld_day', between='Sex',
                              subject='IdRFID', data=d, padjust='fdr_bh')
pg.print_table(posthocs)

## daily and hourly total with separate substances heatmap

### total intake

In [ ]:
data_VDM_total_sub_day_hour = data_final_clean_3s.groupby(['IdRFID', 'Sex', 'substance', 'ld_day', 'zeitgeber',
                                   ])[['intake_ml_kg']].sum().reset_index()

data_VDM_total_sub_day_hour.head()

In [ ]:
d = data_VDM_total_sub_day_hour[data_VDM_total_sub_day_hour['Sex']=='female']
d = d[d['substance']=='Fent20']
groupby = d.groupby(['ld_day', 'zeitgeber'])['intake_ml_kg'].mean().reset_index()


sns.set(font_scale=2)
groupby = groupby.pivot('ld_day', 'zeitgeber', "intake_ml_kg")
plt.figure(figsize=(5,5))
ax = sns.heatmap(groupby, cmap="PuBuGn", linewidths=.25, vmax=10,
                     cbar_kws={'label': 'Total intake (ml/kg)'})

#plt.xlabel('Bottle')
plt.show()

## ID heatmap

In [ ]:
data_VDM_total_sub['Id'] = data_VDM_total_sub['Sex'] + '_' + data_VDM_total_sub['IdRFID'].astype('str')
data_VDM_total_sub.head()

In [ ]:
sns.set(font_scale=2)
groupby = data_VDM_total_sub.pivot('Id', 'substance', "intake_ml_kg")
plt.figure(figsize=(5,30))
ax = sns.heatmap(groupby, cmap="PuBuGn", linewidths=.25, 
                     cbar_kws={'label': 'Total intake (ml/kg)'})

#plt.xlabel('Bottle')
plt.show()

## raster

In [ ]:
data_VDM_start.head()

In [ ]:
mice = ['041AC1620E', '041AC15DE9', '041AC16321', '041AC16A00', ]
data_idivid = data_VDM_start[data_VDM_start['IdRFID']==mice[3]]
data_idivid = data_idivid[data_idivid['ld_day']==4]
colorCodes = ['C{}'.format(i) for i in range(5)]

param = 'VDM_sec_count_running'
data_1 = data_idivid[data_idivid['substance'] == 'water'][param].values
data_2 = data_idivid[data_idivid['substance'] == 'EtOH05'][param].values
data_3 = data_idivid[data_idivid['substance'] == 'EtOH10'][param].values
data_4 = data_idivid[data_idivid['substance'] == 'Fent05'][param].values
data_5 = data_idivid[data_idivid['substance'] == 'Fent20'][param].values

data_array = np.array([data_1, data_2, data_3, data_4, data_5])

# Draw a spike raster plot
plt.figure(figsize=(19,5))
params = {'axes.labelsize': 75,
          'axes.titlesize': 75}
plt.rc('xtick', labelsize=40) 
plt.rc('ytick', labelsize=40) 
plt.rcParams.update(params)
plt.yticks([0,1,2,3,4], 
           ['water', 'EtOH05', 'EtOH10', 'Fent05', 'Fent20'])
plt.eventplot(data_array, color=colorCodes, linelengths = 0.3) 
plt.xlabel('Time (running second count)', fontsize=50)
plt.ylabel('Drinking chamber', fontsize=50)

plt.show()

## preference

#### total by substance

In [ ]:
data_summary_vol = data_VDM_total_sub.set_index(['IdRFID', 'Sex', 
                                  'substance'])[['intake_ml_kg']].unstack(-1).reset_index()

data_summary_vol.columns = ['IdRFID', 'Sex', 
                        'EtOH05', 'EtOH10', 'Fent05', 'Fent20', 'water',]

data_summary_vol.replace(np.nan, 0, inplace=True)

data_summary_vol['total_alcohol_intake'] = data_summary_vol['EtOH05'] + data_summary_vol['EtOH10'] 
data_summary_vol['total_fent_intake'] = data_summary_vol['Fent05'] + data_summary_vol['Fent20'] 

data_summary_vol['overall_alcohol_pref'] = data_summary_vol['total_alcohol_intake'] / (data_summary_vol['total_alcohol_intake'] + data_summary_vol['water'])
data_summary_vol['overall_fent_pref'] = data_summary_vol['total_fent_intake'] / (data_summary_vol['total_fent_intake'] + data_summary_vol['water'])

data_summary_vol['DP_alcohol'] =  ((data_summary_vol['EtOH05']*.05) + (data_summary_vol['EtOH10']*.1))  \
/ (data_summary_vol['EtOH05'] + data_summary_vol['EtOH10'])

data_summary_vol['DP_fent'] =  ((data_summary_vol['Fent05']*.05) + (data_summary_vol['Fent20']*.2))  \
/ (data_summary_vol['Fent05'] + data_summary_vol['Fent20'])

data_summary_vol.replace(np.nan, 0, inplace=True)

data_summary_vol.head()

In [ ]:
g = sns.catplot(x='Sex', y='DP_fent', data=data_summary_vol,  kind='bar', height=5, aspect=1,
            ci=68, palette='PuBuGn')

# map data to stripplot
g.map(sns.stripplot, 'Sex', 'DP_fent', 
      palette=sns.color_palette('PuBuGn')[2:4], dodge=True, alpha=0.6, ec='k', linewidth=1)
plt.ylabel('Fentanly Dose Preference')
plt.ylim(0,.2)
plt.show()

male = data_summary_vol[data_summary_vol['Sex']=='male']['DP_fent']
female = data_summary_vol[data_summary_vol['Sex']=='female']['DP_fent']
stats.ttest_ind(female, male)

#### ld total by substance

In [ ]:
data_summary_vol_day = data_VDM_total_sub_day.set_index(['IdRFID', 'Sex', 'ld_day', 
                                  'substance'])[['intake_ml_kg']].unstack(-1).reset_index()

data_summary_vol_day.columns = ['IdRFID', 'Sex', 'ld_day',
                        'EtOH05', 'EtOH10', 'Fent05', 'Fent20', 'water',]

data_summary_vol_day.replace(np.nan, 0, inplace=True)

data_summary_vol_day['total_alcohol_intake'] = data_summary_vol_day['EtOH05'] + data_summary_vol_day['EtOH10'] 
data_summary_vol_day['total_fent_intake'] = data_summary_vol_day['Fent05'] + data_summary_vol_day['Fent20'] 

data_summary_vol_day['overall_alcohol_pref'] = data_summary_vol_day['total_alcohol_intake'] / (data_summary_vol_day['total_alcohol_intake'] + data_summary_vol_day['water'])
data_summary_vol_day['overall_fent_pref'] = data_summary_vol_day['total_fent_intake'] / (data_summary_vol_day['total_fent_intake'] + data_summary_vol_day['water'])

data_summary_vol_day['DP_alcohol'] =  ((data_summary_vol_day['EtOH05']*.05) + (data_summary_vol_day['EtOH10']*.1))  \
/ (data_summary_vol_day['EtOH05'] + data_summary_vol_day['EtOH10'])

data_summary_vol_day['DP_fent'] =  ((data_summary_vol_day['Fent05']*.05) + (data_summary_vol_day['Fent20']*.2))  \
/ (data_summary_vol_day['Fent05'] + data_summary_vol_day['Fent20'])

data_summary_vol_day.replace(np.nan, 0, inplace=True)
data_summary_vol_day.head()

In [ ]:
g= sns.catplot(x='ld_day', y='DP_fent', data=data_summary_vol_day,  kind='bar', height=5, aspect=1,
            ci=68, hue='Sex', palette='PuBuGn')

# map data to DP_fent
g.map(sns.stripplot, 'ld_day', 'DP_fent', 'Sex', 
      palette=sns.color_palette('PuBuGn')[2:4], dodge=True, alpha=0.6, ec='k', linewidth=1)

#plt.ylim(0,1250)
plt.xlabel('Day')
plt.ylabel('Fentanly Dose Preference')
plt.show()

In [ ]:
dv = 'DP_fent'

#RM using Pingouin
aov = pg.mixed_anova(dv=dv, within='ld_day', between='Sex', subject='IdRFID', data=data_summary_vol_day)
pg.print_table(aov)

#posthocs
posthocs = pg.pairwise_ttests(dv=dv, within='ld_day', between='Sex',
                              subject='IdRFID', data=data_summary_vol_day, padjust='fdr_bh')
pg.print_table(posthocs)

#### daily and hourly total by substance

In [ ]:
data_summary_vol_day = data_VDM_total_sub_day_hour.set_index(['IdRFID', 'Sex', 'ld_day', 'zeitgeber',
                                  'substance'])[['intake_ml_kg']].unstack(-1).reset_index()

data_summary_vol_day.columns = ['IdRFID', 'Sex', 'ld_day', 'zeitgeber',
                        'EtOH05', 'EtOH10', 'Fent05', 'Fent20', 'water',]

data_summary_vol_day.replace(np.nan, 0, inplace=True)

data_summary_vol_day['total_alcohol_intake'] = data_summary_vol_day['EtOH05'] + data_summary_vol_day['EtOH10'] 
data_summary_vol_day['total_fent_intake'] = data_summary_vol_day['Fent05'] + data_summary_vol_day['Fent20'] 

data_summary_vol_day['overall_alcohol_pref'] = data_summary_vol_day['total_alcohol_intake'] / (data_summary_vol_day['total_alcohol_intake'] + data_summary_vol_day['water'])
data_summary_vol_day['overall_fent_pref'] = data_summary_vol_day['total_fent_intake'] / (data_summary_vol_day['total_fent_intake'] + data_summary_vol_day['water'])

data_summary_vol_day['DP_alcohol'] =  ((data_summary_vol_day['EtOH05']*.05) + (data_summary_vol_day['EtOH10']*.1))  \
/ (data_summary_vol_day['EtOH05'] + data_summary_vol_day['EtOH10'])

data_summary_vol_day['DP_fent'] =  ((data_summary_vol_day['Fent05']*.05) + (data_summary_vol_day['Fent20']*.2))  \
/ (data_summary_vol_day['Fent05'] + data_summary_vol_day['Fent20'])


data_summary_vol_day.replace(np.nan, 0, inplace=True)

data_summary_vol_day.head()

In [ ]:
d = data_summary_vol_day[data_summary_vol_day['Sex']=='male']

groupby = d.groupby(['ld_day', 'zeitgeber'])['overall_fent_pref'].mean().reset_index()


sns.set(font_scale=2)
groupby = groupby.pivot('ld_day', 'zeitgeber', "overall_fent_pref")
plt.figure(figsize=(5,5))
ax = sns.heatmap(groupby, cmap="PuBuGn", linewidths=.25, vmax=1,
                     cbar_kws={'label': 'Fentanyl Preference'})

#plt.xlabel('Bottle')
plt.show()